In [276]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from statsmodels.api import qqplot
import scipy.stats as stats

from keras import models
from keras.layers import Dense, Dropout

from pickle import dump


In [277]:
pd.set_option('display.max_columns', None)

In [278]:
# loading the dataset
df = pd.read_csv("./data/kidney_disease.csv")
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,no,no,no,good,no,no,ckd


In [279]:
df.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [281]:
df.isnull().sum()/4

id                 0.00
age                2.25
bp                 3.00
sg                11.75
al                11.50
su                12.25
rbc               38.00
pc                16.25
pcc                1.00
ba                 1.00
bgr               11.00
bu                 4.75
sc                 4.25
sod               21.75
pot               22.00
hemo              13.00
pcv               17.50
wc                26.25
rc                32.50
htn                0.50
dm                 0.50
cad                0.50
appet              0.25
pe                 0.25
ane                0.25
classification     0.00
dtype: float64

In [282]:
df.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.000000,391.000000,388.000000,353.000000,354.000000,351.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000
mean,199.500000,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437
std,115.614301,17.169714,13.683637,0.005717,1.352679,1.099191,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587
min,0.000000,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000
25%,99.750000,42.000000,70.000000,1.010000,0.000000,0.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000
50%,199.500000,55.000000,80.000000,1.020000,0.000000,0.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000
75%,299.250000,64.500000,80.000000,1.020000,2.000000,0.000000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000
max,399.000000,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000


In [283]:
# categorical columns
cat_cols = []
for i in df.columns:
    if df[i].dtype == 'object':
        cat_cols.append(i)

cat_cols

['rbc',
 'pc',
 'pcc',
 'ba',
 'pcv',
 'wc',
 'rc',
 'htn',
 'dm',
 'cad',
 'appet',
 'pe',
 'ane',
 'classification']

In [284]:
cat = pd.DataFrame()
cat['columns'] = cat_cols
cat['missing_vals_in_%'] = [i/4 for i in df[cat_cols].isnull().sum().to_list()]
cat['unique_vals'] = [df[i].unique() for i in cat_cols]
cat

,columns,missing_vals_in_%,unique_vals
0,rbc,38.00,"[nan, normal, abnormal]"
1,pc,16.25,"[normal, abnormal, nan]"
2,pcc,1.00,"[notpresent, present, nan]"
3,ba,1.00,"[notpresent, present, nan]"
4,pcv,17.50,"[44, 38, 31, 32, 35, 39, 36, 33, 29, 28, nan, ..."
5,wc,26.25,"[7800, 6000, 7500, 6700, 7300, nan, 6900, 9600..."
6,rc,32.50,"[5.2, nan, 3.9, 4.6, 4.4, 5, 4.0, 3.7, 3.8, 3...."
7,htn,0.50,"[yes, no, nan]"
8,dm,0.50,"[yes, no, yes, \tno, \tyes, nan]"
9,cad,0.50,"[no, yes, \tno, nan]"


In [285]:
# replacing incorrect values

df.dm.replace(to_replace = {' yes' : 'yes', '\tno' : 'no', '\tyes' : 'yes'}, inplace = True)
df.cad.replace(to_replace = {'\tno' : 'no'}, inplace = True)
df.classification.replace(to_replace = {'ckd\t' : 'ckd'}, inplace = True)

In [286]:
# replacing incorrect values and converting dtype
df.pcv.replace(to_replace={"\t?" : np.nan, '\t43' : '43'}, inplace = True)
df.pcv = df.pcv.astype('Int16')

In [287]:
# replacing incorrect values and converting dtype

df.wc.replace(to_replace={'\t6200' : '6200', '\t8400' : 8400, '\t?' : np.nan, 'nan' : np.nan, float(np.nan) : np.nan}, inplace = True)
df['wc'] = np.floor(pd.to_numeric(df['wc'], errors='coerce')).astype('Int64')

In [288]:
# replacing incorrect values and converting dtype

df.rc.replace(to_replace = {'\t?' : np.nan}, inplace = True)
df.rc = df.rc.astype('float64')

In [289]:
df[cat_cols]

,rbc,pc,pcc,ba,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,NaN,normal,notpresent,notpresent,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,NaN,normal,notpresent,notpresent,38,6000,NaN,no,no,no,good,no,no,ckd
2,normal,normal,notpresent,notpresent,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,normal,abnormal,present,notpresent,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,normal,normal,notpresent,notpresent,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,normal,normal,notpresent,notpresent,47,6700,4.9,no,no,no,good,no,no,notckd
396,normal,normal,notpresent,notpresent,54,7800,6.2,no,no,no,good,no,no,notckd
397,normal,normal,notpresent,notpresent,49,6600,5.4,no,no,no,good,no,no,notckd
398,normal,normal,notpresent,notpresent,51,7200,5.9,no,no,no,good,no,no,notckd


In [290]:
cat_cols = []
for i in df.columns:
    if df[i].dtype == 'object':
        cat_cols.append(i)

# we use label encoding for all the categorical columns except null values
encoders = {} #dictionary for saving instance of each encoder for each categorical columns
encoder = LabelEncoder()
for cols in cat_cols:
    without_null = df[cols][df[cols].notnull()]
    df[cols] = pd.Series(encoder.fit_transform(without_null), index = without_null.index)
    encoders[cols] = encoder

In [291]:
df[cat_cols]

,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,classification
0,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
395,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
396,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
397,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
398,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [292]:
# now we will use iterative-imputer to fill missing values
num_cols = [i for i in df.columns if i not in cat_cols]

imp_num = IterativeImputer(estimator=RandomForestRegressor(),
                               initial_strategy='mean',
                               max_iter=10, random_state=0)
imp_cat = IterativeImputer(estimator=RandomForestClassifier(), 
                               initial_strategy='most_frequent',
                               max_iter=10, random_state=0)

df[num_cols] = imp_num.fit_transform(df[num_cols])
df[cat_cols] = imp_cat.fit_transform(df[cat_cols])




c:\Users\RAJESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Users\RAJESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [293]:
df.isnull().sum()  # all the null values are imputed.

id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64

In [294]:
df.classification.value_counts()

0.0    250
1.0    150
Name: classification, dtype: int64

In [295]:
x = df.drop(['classification', 'id'], axis = 1)
y = df['classification']
xtrain, xtest, ytrain, ypred = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [296]:
x

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane
0,48.0,80.0,1.020,1.0,0.0,1.0,1.0,0.0,0.0,121.00,36.0,1.2,128.44,4.033,15.4,44.0,7800.0,5.200,1.0,1.0,0.0,0.0,0.0,0.0
1,7.0,50.0,1.020,4.0,0.0,0.0,1.0,0.0,0.0,152.86,18.0,0.8,125.66,3.897,11.3,38.0,6000.0,4.391,0.0,0.0,0.0,0.0,0.0,0.0
2,62.0,80.0,1.010,2.0,3.0,1.0,1.0,0.0,0.0,423.00,53.0,1.8,117.64,3.956,9.6,31.0,7500.0,3.673,0.0,1.0,0.0,1.0,0.0,1.0
3,48.0,70.0,1.005,4.0,0.0,1.0,0.0,1.0,0.0,117.00,56.0,3.8,111.00,2.500,11.2,32.0,6700.0,3.900,1.0,0.0,0.0,1.0,1.0,1.0
4,51.0,80.0,1.010,2.0,0.0,1.0,1.0,0.0,0.0,106.00,26.0,1.4,122.52,3.685,11.6,35.0,7300.0,4.600,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,140.00,49.0,0.5,150.00,4.900,15.7,47.0,6700.0,4.900,0.0,0.0,0.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,75.00,31.0,1.2,141.00,3.500,16.5,54.0,7800.0,6.200,0.0,0.0,0.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,100.00,26.0,0.6,137.00,4.400,15.8,49.0,6600.0,5.400,0.0,0.0,0.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,114.00,50.0,1.0,135.00,4.900,14.2,51.0,7200.0,5.900,0.0,0.0,0.0,0.0,0.0,0.0


In [260]:
xtrain.shape

(320, 24)

In [261]:
# creating a neural network

model = models.Sequential() 
model.add(Dense(activation = "relu", input_dim = 24,  
                     units = 24, kernel_initializer = "uniform")) 
model.add(Dense(activation = "relu", units = 14,  
                     kernel_initializer = "uniform")) 
model.add(Dense(activation = "sigmoid", units = 1,  
                     kernel_initializer = "uniform")) 
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy',  
                   metrics = ['accuracy'] )

In [262]:
# tranining the model
model.fit(xtrain , ytrain , batch_size = 8 ,epochs = 100 ) 

Epoch 1/100


40/40 [==============================] - 8s 7ms/step - loss: 0.7065 - accuracy: 0.5469
Epoch 2/100
40/40 [==============================] - 0s 8ms/step - loss: 0.6560 - accuracy: 0.6094
Epoch 3/100
40/40 [==============================] - 0s 8ms/step - loss: 0.6427 - accuracy: 0.6094
Epoch 4/100
40/40 [==============================] - 0s 6ms/step - loss: 0.6490 - accuracy: 0.6313
Epoch 5/100
40/40 [==============================] - 0s 6ms/step - loss: 0.6347 - accuracy: 0.6313
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.6383 - accuracy: 0.6344
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.6191 - accuracy: 0.6344
Epoch 8/100
40/40 [==============================] - 0s 4ms/step - loss: 0.6038 - accuracy: 0.6344
Epoch 9/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5798 - accuracy: 0.6531
Epoch 10/100
40/40 [==============================] - 0s 3ms/step - loss: 0.5565 - accuracy: 0.7437
Epoch 1

In [263]:
ypred = model.predict(xtest)

3/3 [==============================] - 1s 14ms/step


In [264]:
np.set_printoptions(suppress=True)   

In [273]:
ypred

array([[0.9865552 ],
       [0.0000013 ],
       [0.9193477 ],
       [0.99149036],
       [0.00000002],
       [0.9964274 ],
       [0.00002099],
       [0.81602085],
       [0.05740219],
       [0.99373823],
       [0.00007415],
       [0.99471146],
       [0.03690498],
       [0.93490523],
       [0.97791266],
       [0.03664492],
       [0.99799913],
       [0.0306159 ],
       [0.00001693],
       [0.00000885],
       [0.0000002 ],
       [0.00000001],
       [0.9966765 ],
       [0.2568827 ],
       [0.00021441],
       [0.00007849],
       [0.9887109 ],
       [0.00000907],
       [0.9954451 ],
       [0.0000543 ],
       [0.00000202],
       [0.07842999],
       [0.00000666],
       [0.9633982 ],
       [0.00000868],
       [0.00021144],
       [0.00000262],
       [0.9922226 ],
       [0.9901263 ],
       [0.00006788],
       [0.98969203],
       [0.00000026],
       [0.00000896],
       [0.07546277],
       [0.99591184],
       [0.0005317 ],
       [0.9898695 ],
       [0.954

In [275]:
# dumping the neural network
dump(model, open(r"./models_dump/kidney_pred_models/model.pkl", 'wb'))

# dumping the imputers and 'num_cols' and 'cat_cols' variables
dump(imp_num, open(r"./models_dump/kidney_pred_models/imputers/imp_num.pkl", 'wb'))
dump(imp_cat, open(r"./models_dump/kidney_pred_models/imputers/imp_cat.pkl", 'wb'))
dump(num_cols, open(r"./models_dump/kidney_pred_models/imputers/num_cols.pkl", 'wb'))
dump(cat_cols, open(r"./models_dump/kidney_pred_models/imputers/cat_cols.pkl", "wb"))

# dumping the encoders
dump(encoders, open(r"./models_dump/kidney_pred_models/encoders/encoders.pkl", 'wb'))